# Competition Metric

In [1]:
import numpy as np
import pandas as pd
import sys

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

sys.path.append('../../')
from scoring.event_detection_matrix import competition_score

## Daten laden

In [2]:
series_id = '0a96f4993bd7'

In [3]:
events = pd.read_csv('../../data/raw/train_events.csv')
events = events[events.series_id == series_id]
events['step'] = events['step'].astype(int)
series = pd.read_parquet('../../data/raw/train_series.parquet', filters=[('series_id', '=', series_id)])

## Daten vorbereiten

In [4]:
series['timestamp'] = pd.to_datetime(series['timestamp']).apply(lambda t: t.tz_localize(None))
events['timestamp'] = pd.to_datetime(events['timestamp']).apply(lambda t: t.tz_localize(None))

In [5]:
series

,series_id,step,timestamp,anglez,enmo
0,0a96f4993bd7,0,2018-05-03 10:30:00,3.272400,0.0386
1,0a96f4993bd7,1,2018-05-03 10:30:05,0.073100,0.0663
2,0a96f4993bd7,2,2018-05-03 10:30:10,-2.456300,0.0408
3,0a96f4993bd7,3,2018-05-03 10:30:15,-5.466900,0.0820
4,0a96f4993bd7,4,2018-05-03 10:30:20,-10.008800,0.0788
...,...,...,...,...,...
256855,0a96f4993bd7,256855,2018-05-18 07:14:35,-62.247898,0.0153
256856,0a96f4993bd7,256856,2018-05-18 07:14:40,-62.449299,0.0085
256857,0a96f4993bd7,256857,2018-05-18 07:14:45,-61.691601,0.0095
256858,0a96f4993bd7,256858,2018-05-18 07:14:50,-60.325699,0.0259


In [6]:
start = '2018-05-04 15:00:00'
end = '2018-05-05 15:00:00	'
start_step = series[series.timestamp == start].iloc[0].step
end_step = series[series.timestamp == end].iloc[0].step

In [7]:
partial_series = series[start_step:end_step]

In [8]:
events = events[(events.step > start_step) & (events.step < end_step)]
events['score'] = 1
events

,series_id,night,event,step,timestamp,score
450,0a96f4993bd7,2,onset,26460,2018-05-04 23:15:00,1
451,0a96f4993bd7,2,wakeup,33972,2018-05-05 09:41:00,1


## Analyse

In [22]:
events_prediction = events.copy()
diff_steps = 24
events_prediction['step'] = events_prediction['step'] + diff_steps
events_prediction['timestamp'] = events_prediction['timestamp'] + pd.Timedelta(seconds=diff_steps * 5)

In [23]:
competition_score(events, events_prediction)

0.9

In [24]:
fig = px.line(partial_series, x='timestamp', y='enmo')
for index, row in events.iterrows():
    fig.add_vline(x=row.timestamp, line_width=1, line_color='red')
    
for index, row in events_prediction.iterrows():
    fig.add_vline(x=row.timestamp, line_width=1, line_dash='dash', line_color='red')


fig.update_layout(legend_title="Events", width=1100, height=150, xaxis_title='', margin=dict(l=0,r=0,b=0,t=0))

C:\Users\timon\.conda\envs\ai-challenge\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

